# **Grouping based on Geo Data**
37명 4개씩 상점 분할하기
- **Total** : 151 개
- 1 개의 상호가 2개의 점포를 소유중 **(1개로 포함하여 작업)**

## **1 Loading the table**
간단한 내용 수정하기

In [1]:
import pandas as pd
file_name1 = "./data/ourtown_store_geo3.csv"
file_name2 = "./data/ourtown_store_geo2.csv"
table = pd.read_csv(file_name1).loc[:,['name', 'lon', 'lat','type','category','label']]
table = table.join(pd.read_csv(file_name2).loc[:,["주소","address","addressroad"]])
table.head(2)

,name,lon,lat,type,category,label,주소,address,addressroad
0,모든홍삼,126.938397,37.485887,기타판매,itemsales,7,"은천로 28, 봉일프라자 나-120",서울특별시 관악구 은천로 28 봉일프라자,서울특별시 관악구 봉천동 951-25 봉일프라자
1,종로떡집,126.939588,37.487197,떡집,food,26,국회단지길 21,서울특별시 관악구 국회단지길 21,서울특별시 관악구 봉천동 636-110


## **2 Checking the Duplicated Values**
동일한 주소값 필터링 (Marker 에서 중복내용 미확인으로 인한 오류방지)
```
%whos | grep DataFrame
```

In [2]:
# Duplicated Address FilteringS
from collections import Counter
_count_address = dict(Counter([_ for _ in table['address']]))
_count_address = {k:str(v) for k, v in _count_address.items() if v != 1}
_single_address = [_ for _ in table['address'] if _ not in list(_count_address.keys())]
len(_count_address), len(_single_address), _count_address

(11,
 125,
 {'서울특별시 관악구 남부순환로214길 29': '2',
  '서울특별시 관악구 난곡로24길 6': '3',
  '서울특별시 관악구 봉천로 465': '2',
  '서울특별시 관악구 난향길 14': '4',
  '서울특별시 관악구 난향길 12': '2',
  '서울특별시 관악구 난곡로26길 7': '2',
  '서울특별시 관악구 난곡로24길 29': '3',
  '서울특별시 관악구 신림로65길 31': '2',
  '서울특별시 관악구 신림로11길 20': '2',
  '서울특별시 관악구 대학5길 21': '2',
  '서울특별시 관악구 호암로22길 69 신국고시원': '2'})

In [3]:
# New Building the Counting Table
_set_address = list(_count_address.keys()) + _single_address
def filter_table(table, address):
    """address 를 기준으로 DataFrame Filtering 함수"""
    _count_df = [table[table['address']==_].iloc[0,:].to_list()  for _ in address]
    _count_df = pd.DataFrame(_count_df)
    _count_df.columns = table.columns
    _count_df = _count_df.loc[:,["lon","lat","address"]]
    return _count_df
_count_df = filter_table(table, _set_address)
_count_df.tail(2)

,lon,lat,address
134,126.965957,37.476538,서울특별시 관악구 남부순환로247길 3
135,126.921094,37.468968,서울특별시 관악구 난곡로24길 32


In [4]:
# Adding the Counting Table
_count_list = []
for _ in _count_df['address']:
    if _ in list(_count_address.keys()):
        _count_list.append(_count_address[_])
    else: _count_list.append("1")
        
_count_df.insert(0,'count', _count_list)
_count_df.tail(2)

,count,lon,lat,address
134,1,126.965957,37.476538,서울특별시 관악구 남부순환로247길 3
135,1,126.921094,37.468968,서울특별시 관악구 난곡로24길 32


## **3 Adding the Index & Addresses**
데이터 위치별 중복되는 상호명 갯수 Popup 으로 표시하기

In [5]:
result = {}
for _ in range(len(table)):
    _df     = table.iloc[_,:]
    name    = f"{_} : {_df['name']}"
    address = _df['address']
    if address not in list(result.keys()): result[address] = [name]
    else: result[address].append(name)
result = [result[_]  for _ in _count_df['address']]
_count_df.insert(4, 'names', result)
_count_df.tail(2)

,count,lon,lat,address,names
134,1,126.965957,37.476538,서울특별시 관악구 남부순환로247길 3,[146 : 마노아김밥]
135,1,126.921094,37.468968,서울특별시 관악구 난곡로24길 32,[150 : 컴플러스 컴퓨터]


<br />

# **Folium Map**
**[Folium Tutorial](https://python-visualization.github.io/folium/quickstart.html#Markers)**

## **1 Visualization**
Function building

In [6]:
import folium
from folium import plugins
def folium_map(table, title='name', tooltip="name"):
    # 'white', 'beige', 
    color_map = ['gray','red', 'purple', 'lightred',  'black',  'green', 'pink', 'darkblue','lightgray', 
        'blue', 'darkgreen', 'lightblue', 'darkpurple', 'darkred', 'lightgreen', 'orange', 'cadetblue'] * 10
    color_map = {_:color_map[no]  for no, _ in enumerate(sorted(set(table[title].to_list())))}
    table = table.reset_index(drop=True)
    _map = folium.Map(  # tiles = 'Stamen Watercolor',
        [table['lat'].mean(), table['lon'].mean()], zoom_start = 14)
    for _ in range(len(table)):
        color = color_map[table.loc[_,title]]
        icon_style = plugins.BeautifyIcon(
            number = str(table.loc[_,title]),
            icon_shape = "marker", inner_icon_style = 'margin=top:0;',
            border_color = color, text_color = color, border_width=2,
        )
        folium.Marker([table.loc[_,'lat'], table.loc[_,'lon']], # opacity = 0.3,
            tooltip = table.loc[_,tooltip], icon = icon_style).add_to(_map)
    return _map

# _map = folium_map(_count_df, title='count', tooltip='names')
# _map.save('./Web/test_duplicate.html')

<br />

# **Classification The Marker DataSet**
앞에서 작업했던 내용을 바탕으로 Marking 작업 실시
## **1 Marking the Number**
Function building

In [7]:
label_index = [
    # <<< part 1 >>>
    # 난향동, 난곡동 지역분할
    [70,81,84,86], [83,95,85,49], # 난향동 9개 분할
    [82,101,104,147],             # 난향동(82) 1개, 난곡동 집중 3개
    [36,51,100,103], [88,89,94,99], [87,93,96,150], # 난곡동 18개 분할
    [91,97,98,105], [45,90,92,102],
    
    # 녹두거리 분할하기 
    [125, 122, 128, 143], [39, 79, 31, 43, 44], #(44번 스터디 카페 제외)
    [72, 133, 53, 144], [69, 121, 141, 127], [124, 129, 130, 136],
    
    #<<< part 2 >>>
    # 녹두거리 분할하기 2
    [123, 132, 126, 137], [76, 138, 134, 142], [139, 131, 135, 140],
    
    # 신림로 중앙 넓은지역 4개
    [10,16,17,73], 
    
    # 신림동 뒷골목 분할
    [116,148, 14,57], # 5개 지역 : 겹치는 장소 1곳 + 주변장소
    [67, 114, 50, 56], [30,32,62,110], [112,115,117,59],
    
    # 국회단지길 주변
    [0,1,2,24,26],  # 5개 지역 (인근지역)
    [3,21,22,23],   
    
    #<<< part 3 >>>
    # 서울대 입구역 주변
    [19,46,113,149,109],    

    # 사당역에서 이어지는 동쪽 끝 지역
    [80, 118, 65, 66],  
    [13, 40, 146, 33], [38, 35, 27, 106], [71, 145, 120, 52], 
    
    # 봉천사거리 서쪽지역
    [8, 12, 37, 119], [111, 48, 11, 64], [41,25,78,28], [4,29,47,58],
    [77, 42, 60, 15], [54, 34, 61, 20], [5, 6, 7, 75], 

    # 남부순환로 가게들 주변
    [9,18,74,108], [55,63,68,107],
]

In [8]:
_temp = []
for _ in label_index:
    _temp += _ 
rest_index = [_ for _ in range(151)  if _ not in _temp]
print(len(rest_index), len(label_index)) #, rest_index
Counter(table.iloc[rest_index,:]['address'])

0 37


Counter()

In [9]:
# Unique Validation Check
from collections import Counter
_result = []
for _ in label_index:
    _result += _
_result = [f"{k} : {v}"  for k,v in dict(Counter(_result)).items() if v != 1]
if len(_result) > 0: print(", ".join(_result))
else: print("Unique Dataset validation check passed!")

Unique Dataset validation check passed!


## **2 Filtering The Numbers**
table_marker : DataFrame 

In [10]:
# Marking "Orginal Table Index : Count Table Index"
_result, _result_temp = {}, {}
for index_number, _list in enumerate(_count_df['names'].to_list()):
    _result_temp[str(index_number)] = []
    for _ in _list: # split count table numbers
        number = _.split(":")[0].strip()
        _result_temp[str(index_number)].append(number)
for count_index, table_index in _result_temp.items():
    for _ in table_index: 
        if _ not in _result.keys(): _result[_] = count_index
count_table_index = _result
len(count_table_index) == len(table)

True

In [11]:
# building the Marking Number
from collections import Counter
table_marker = table.loc[:, ["name","lon","lat","type","category","label","주소","address"]]
for no, _list in enumerate(label_index):
    for _ in _list:
        table_marker.loc[_,'label'] = f"c{no+1}"
table_marker['label'] = list(map(lambda x : str(x), table_marker['label']))
_temp_values = [count_table_index[str(_)] for _ in range(len(table_marker))]
table_marker.insert(6, 'index_countable', _temp_values)
count_dict = dict(Counter([_ for _ in table_marker['label'] if str(_).find("c") != -1]))
", ".join([f"{k}:{v}개" for k,v in count_dict.items()  if v!=4])

'c22:5개, c24:5개, c10:5개'

## **3 Marking the Map**
분류 작업이 종료된 내용 필터링

In [12]:
table_marker.head(2)

,name,lon,lat,type,category,label,index_countable,주소,address
0,모든홍삼,126.938397,37.485887,기타판매,itemsales,c22,11,"은천로 28, 봉일프라자 나-120",서울특별시 관악구 은천로 28 봉일프라자
1,종로떡집,126.939588,37.487197,떡집,food,c22,12,국회단지길 21,서울특별시 관악구 국회단지길 21


In [13]:
import folium
from folium import plugins
def folium_map(table, title='label', tooltip="name", opacity=0.3):
    # 'white', 'beige', 'pink', 'lightgray','lightgreen', 'lightblue','lightred',  
    # color_map = ['gray','red', 'purple', 'lightred',  'black',  'green',  'darkblue', 
    #    'blue', 'darkgreen',  'darkpurple', 'darkred', 'orange', 'cadetblue'] * 10
    color_map = ['red', 'blue', 'purple', 'orange', 'green', 'black'] * 10
    color_map = {_:color_map[no]  for no, _ in enumerate(sorted(set(table[title].to_list())))}
    table = table.reset_index(drop=True)
    _map = folium.Map(  # tiles = 'Stamen Watercolor',
        [table['lat'].mean(), table['lon'].mean()], zoom_start = 14)
    for _ in range(len(table)):
        if _ != 44: # 중복기재로 인한 제외 인덱스 데이터 
            color = color_map[table.loc[_,title]]
            title_name = str(_) + "," + str(table.loc[_,title])
            # Label 에서 "c" 가 붙었는지 내용 확인하기
            if title_name.find('c') != -1:
                title_name    = title_name.split(",")[-1]
                opacity_value = opacity  # 이미 분류작업이 끝난경우
            else: 
                title_name    = title_name.split(",")[0]
                opacity_value = 1
            icon_style = plugins.BeautifyIcon(
                number = title_name,
                icon_shape = "marker", inner_icon_style = 'margin-top:0;',
                border_color = color, text_color = color, border_width=2,
            )
            folium.Marker([table.loc[_,'lat'], table.loc[_,'lon']], opacity = opacity_value,
                tooltip = str(_)+":"+table.loc[_,tooltip]+":"+table.loc[_,'category'],
                icon = icon_style).add_to(_map)
    return _map

_map = folium_map(table_marker, title='label', tooltip='name', opacity=1)
_map.save('web/test.html')

<br />

# **Finished the Job**
작업한 내용 정리하기

## **1 Building the DataFrame Table**
Function building

In [14]:
part_index = {
    "1th" : [ # Green Area
        # <<< part 1 >>>
        # 난향동, 난곡동 지역분할
        [70,81,84,86], [83,95,85,49], # 난향동 9개 분할
        [82,101,104,147],             # 난향동(82) 1개, 난곡동 집중 3개
        [36,51,100,103], [88,89,94,99], [87,93,96,150], # 난곡동 18개 분할
        [91,97,98,105], [45,90,92,102],

        # 녹두거리 분할하기 
        [125, 122, 128, 143], [39, 79, 31, 43, 44], #(44번 스터디 카페 제외)
        [72, 133, 53, 144], [69, 121, 141, 127], [124, 129, 130, 136],        
    ],
    "2th" : [
        #<<< part 2 >>>
        # 녹두거리 분할하기 2
        [123, 132, 126, 137], [76, 138, 134, 142], [139, 131, 135, 140],

        # 신림로 중앙 넓은지역 4개
        [10,16,17,73], 
        
        # 국회단지길 북쪽지역
        [0,1,2,24,26],  # 5개 지역 (인근지역)

        # 신림동 뒷골목 분할
        [116,148, 14,57],
        [67, 114, 50, 56], [30,32,62,110], [112,115,117,59],[71, 145, 120, 52],
        [77, 42, 60, 15], [54, 34, 61, 20],
    ],
    "3th" : [ # Blue Area
        # 봉천사거리 서쪽지역
        [8, 12, 37, 119], [111, 48, 11, 64], [41,25,78,28], [4,29,47,58],

        
        #<<< part 3 >>>
        # 사당역에서 이어지는 동쪽 끝 지역
        [80, 118, 65, 66],  
        [13, 40, 146, 33], [38, 35, 27, 106], 

        # 국회단지길 북쪽지역
        [3,21,22,23], [5, 6, 7, 75], 
        
        # 서울대 입구역 주변
        [19,46,113,149,109], # 5개 지역

        # 남부순환로 가게들 주변
        [9,18,74,108], [55,63,68,107],        
    ],
}

In [15]:
_result = {}
for _ in part_index.keys():
    _result[_] = []
    for _list in part_index[_]:
        _result[_] += _list
    counts = len(_result[_])
    print(f"{_} : {counts}")

# 기존 테이블에 관리자 번호 추가하기
table_marker['observer'] = table_marker['label']
for _oberver in _result.keys():
    for _index in _result[_oberver]:
        table_marker.loc[_index, 'observer'] = _oberver

# 관리자 및 label 추가한 내용으로 새롭게 테이블 정리하기
table_marker_fin = table_marker.loc[:, 
    ["observer", "label","name", "lat", "lon", '주소', 
     'address',  'type', 'category']].reset_index()

_numbers = list(map(lambda x: int(x[1:]) ,table_marker_fin['label']))
table_marker_fin['label'] = [f"{_:02}" for _ in _numbers]
table_marker_fin = table_marker_fin.sort_values(['observer', 'label']).reset_index(drop=True)
table_marker_fin = table_marker_fin.drop([39]).reset_index(drop=True)
# table_marker_fin.to_csv("./data/ourtown_store_geo_fin.csv", index=None)
table_marker_fin.head(2)

1th : 53
2th : 49
3th : 49


,index,observer,label,name,lat,lon,주소,address,type,category
0,70,1th,01,카페비온,37.462847,126.917727,난향길 24,서울특별시 관악구 난향길 24 현경빌딩,카페,food
1,81,1th,01,김현애보습학원,37.462919,126.918079,"난향길 14, 2층",서울특별시 관악구 난향길 14,학원,bizspace


## **2 Marking the Map**
분류 작업이 종료된 내용 필터링

In [16]:
import folium
from folium import plugins
def folium_map(table, title='label', tooltip="name", opacity=0.3):
    # 'white', 'beige', 'pink', 'lightgray','lightgreen', 'lightblue','lightred',  
    # color_map = ['gray','red', 'purple', 'lightred',  'black',  ,  'darkblue', 
    #    'blue', 'darkgreen',  'darkpurple', 'darkred', 'orange', 'cadetblue'] * 10
    color_map = ['red', 'blue', 'purple', 'orange', 'green', 'black', 'green'] * 30
    color_map = {_:color_map[no]  for no, _ in enumerate(sorted(set(table[title].to_list())))}
    color_observer_map = {"1th":"darkblue", "2th":"orange", "3th":"darkred"}
    table = table.reset_index(drop=True)
    _map = folium.Map(  # tiles = 'Stamen Watercolor',
        [table['lat'].mean(), table['lon'].mean()], zoom_start = 14)
    for _ in range(len(table)):
        if _ != 44: # 중복기재로 인한 제외 인덱스 데이터 
            color = color_map[table.loc[_,title]]
            color_border = color_observer_map[table.loc[_, "observer"]] 
            title_name = str(_) + "," + str(table.loc[_,title])
            if title_name.find('c') != -1:  # Label 에서 "c" 가 붙어있는 경우
                title_name    = title_name.split(",")[-1]
                opacity_value = opacity     # 이미 분류작업이 끝난경우
            else:  # Label
                title_name    = title_name.split(",")[0]
                opacity_value = 1
            icon_style = plugins.BeautifyIcon(
                number = table.loc[_,title], # str(table.loc[_,'index']),
                icon_shape = "marker", inner_icon_style = 'margin=top:1;',
                border_color = color_border, text_color = color, border_width=2,
            )
            folium.Marker([table.loc[_,'lat'], table.loc[_,'lon']], opacity = opacity_value,
                tooltip = str(_)+":"+table.loc[_,tooltip]+":"+table.loc[_,'category'],
                icon = icon_style).add_to(_map)
    return _map

_map = folium_map(table_marker_fin, title='label', tooltip='name', opacity=0.2)
# _map.save('./Web/gkart_v1.html')

## **2 Fixing and Saving The Table**
테이블 내용 확인

In [17]:
# folium_map(table_marker_fin[table_marker_fin['observer'] == "2th"],
# title='label', tooltip='name', opacity=1)

In [18]:
table_marker_fin.head(2) #.to_csv("./data/ourtown_store_geo_fin.csv", index=None)
# table_marker_fin.to_csv("./data/ourtown_store_geo_fin.csv", index=None)

,index,observer,label,name,lat,lon,주소,address,type,category
0,70,1th,01,카페비온,37.462847,126.917727,난향길 24,서울특별시 관악구 난향길 24 현경빌딩,카페,food
1,81,1th,01,김현애보습학원,37.462919,126.918079,"난향길 14, 2층",서울특별시 관악구 난향길 14,학원,bizspace


In [19]:
table_source = pd.read_csv('./data/ourtown_store.csv')
table_source.head(2)

,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,기타
0,1,모든홍삼,주옥자,010-7908-0324,"은천로 28, 봉일프라자 나-120",기타판매,은천동,X,X,개별점포,NaN
1,2,종로떡집,김기철,010-4606-7280,국회단지길 21,떡집,은천동,X,X,개별점포,NaN


In [20]:
table_marker_fin2 = table_marker_fin.join(pd.DataFrame([
    table_source.loc[_,["점주명", "연락처"]]  for _ in table_marker_fin['index']]))
table_marker_fin2 = table_marker_fin2.loc[:,[
    "observer","label","name","점주명","연락처",'lat','lon','address','주소',"type"]]
table_marker_fin2.to_csv("./data/ourtown_store_geo_fin.csv", index=None)
table_marker_fin2.tail(2)

,observer,label,name,점주명,연락처,lat,lon,address,주소,type
148,3th,37,임서하헤어마벨,박영숙,010-2670-8392,37.481842,126.941729,서울특별시 관악구 장군봉1길 29,장군봉1길 29,미용업
149,3th,37,신현선무관,장귀길,010-8249-0100,37.481743,126.942606,서울특별시 관악구 장군봉1길 45,"장군봉1길 45, 2층",학원


## **3 VisualiZation**
내용의 추가 및 중복데이터 수정한 Map 출력하기

In [21]:
import pandas as pd
table_fin = pd.read_csv("./data/ourtown_store_geo_fin2.csv")
table_fin.head(2)

,master,atrist,title,name,phone,lat,lon,address,addressraw,type
0,1th,1,카페비온,주옥자,010-7908-0324,37.462847,126.917727,서울특별시 관악구 난향길 24 현경빌딩,난향길 24,카페
1,1th,1,김현애보습학원,김기철,010-4606-7280,37.462919,126.918079,서울특별시 관악구 난향길 14,"난향길 14, 2층",학원


In [23]:
import folium
from folium import plugins
def folium_map(table, title='label', name = "name", tooltip="name", opacity=0.3):
    _map  = folium.Map([table['lat'].mean(), table['lon'].mean()], zoom_start = 14)
    color_map = ['red', 'blue', 'purple', 'orange', 'green', 'black'] * 100
    color_observer_map = {"1th":"darkblue", "2th":"darkgreen", "3th":"darkred"}
    for _ in range(len(table)):
        _ = int(_)
        _table = table.iloc[_,:]
        color = color_map[int(_table[title])]
        color_border  = color_observer_map[_table["observer"]]
        icon_style = plugins.BeautifyIcon(
            number = str(_table[title]),
            icon_shape = "marker", inner_icon_style = 'margin=top:1;',
            border_color = color_border, text_color = color, border_width=4,
        )
        folium.Marker([_table['lat'], _table['lon']], opacity = opacity,
            tooltip = _table[tooltip] + " : " + _table['addressraw'],
            icon = icon_style).add_to(_map)
    return _map

_map = folium_map(table_fin, title='title', tooltip='titla', opacity=1)
_map.save('web/gkart.html')

ValueError: invalid literal for int() with base 10: '카페비온'

In [ ]:
_map

In [ ]:
table_fin.iloc[10,:]

In [ ]:
table_fin.iloc[1,:]

In [ ]:
_map